## This notebook covers some of the general code for extracting the data from OMOP

### But note that for improved specificity, some of the data cleaning and mappings with external datasets may be specific to each individual care site.

In [ ]:
# imports
import pandas as pd
import pyodbc
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
import sqlalchemy
import urllib

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
np.set_printoptions(threshold=500)
basedir = os.getcwd()

In [ ]:
# Connect to database
con = pyodbc.connect("{EHR_CONNECTION_STRING}")

In [ ]:
%run ehr_helper_functions.py

In [ ]:
# get information for all OMOP patients
snapshot_date = '2022/01/01'
original_emr_pt_table = 'emr_patient_table'
sqlstring = f"""SELECT P.person_id, PDD.Sex, PDD.RaceEthnicity, 
                       P.birth_datetime, VOE.n_visits, VOE.min_date, VOE.max_date, 
                       DATEDIFF(year, P.birth_datetime, '{snapshot_date}') as e_age
                FROM person P 
                LEFT JOIN {original_emr_pt_table} PDD
                    ON PDD.pt_key = P.person_source_value
                LEFT JOIN (  SELECT VO.person_id, COUNT(DISTINCT VO.visit_occurrence_id) n_visits, 
                             MIN((CASE WHEN VO.visit_start_date <= VO.visit_end_date THEN VO.visit_start_date
                                  ELSE VO.visit_end_date END)) as min_date, 
                             MAX((CASE WHEN VO.visit_start_date >= VO.visit_end_date THEN VO.visit_start_date
                                  ELSE VO.visit_end_date END)) as max_date
                             FROM visit_occurrence VO
                             GROUP BY VO.person_id) VOE
                    ON VOE.person_id = P.person_id"""
cohort_info = pd.read_sql_query(sqlstring, con)

In [ ]:
# Map diagnoses information and updated onset dates with UCSF Memory and Aging Center database
cohortpts = pd.read_csv('data/example_MAC_pts.csv')
# columns: person_id, dx, firstdx_date

In [ ]:
# Get concept mappings 
dem_concepts = get_string_condition_related_descendants('dementia')
ad_concepts = get_string_condition_related_descendants('alzheimer')

# get exclusion concepts
neurodeg_concepts = get_icd10_related_descendants('G[123]%') 
nonexclusion_concepts = [45571793, 35207345, 376337, 73754, 45586281, 435802]
neurodeg_concepts = neurodeg_concepts[~neurodeg_concepts.ultconcept.isin(nonexclusion_concepts)]

# Get dementia drugs
demdrug_concepts = get_ATC_related_descendants('N06D')

In [ ]:
# Get Index time for Alzheimers Disease
# this is the diagnosis of any dementia OR MCI 
# OR prescription of donepezil, memantine, galantamine, and other anti-dementia drug in the N06D ATC category
sqlstring ="""
        SELECT CO.person_id, MIN(CO.condition_start_date) as min_date, cc.concept_name
        FROM condition_occurrence CO   
        LEFT JOIN concept cc
            ON CO.condition_concept_id = cc.concept_id     
        WHERE CO.condition_concept_id in ({}) and CO.person_id in ({})
        group by CO.person_id, cc.concept_name
        """.format(",".join(np.concatenate((ad_concepts.concept_id_2.astype(str).unique(), 
                                            dem_concepts.concept_id_2.astype(str).unique()))), 
                  ",".join(cohortpts.person_id.astype(str).values))
ad_cond = pd.read_sql_query(sqlstring, con)

sqlstring ="""
        SELECT DE.person_id, MIN(DE.drug_exposure_start_date) as min_date, cc.concept_name
        FROM drug_exposure DE
        LEFT JOIN concept cc
            ON DE.drug_concept_id = cc.concept_id     
        WHERE DE.drug_concept_id in ({}) and DE.person_id in ({})
        group by DE.person_id, cc.concept_name
        """.format(",".join(demdrug_concepts.ultconcept.astype(str).unique()), 
                   ",".join(cohortpts.person_id.astype(str).values))
ad_drug = pd.read_sql_query(sqlstring, con)

# get overall minimum date within multiple entries per patient
ad_conddrug = ad_cond.append(ad_drug).sort_values(['person_id','min_date'])
ad_conddrug = ad_conddrug.merge(cohortpts, on = 'person_id', how = 'left')
ad_conddrug['min_date'] = pd.to_datetime(ad_conddrug['min_date'])
ad_conddrug['mindaterow'] = ad_conddrug[['min_date','firstdx_date']].min(axis=1)
ptmindate = ad_conddrug.groupby('person_id')['mindaterow'].min().to_frame().rename({'mindaterow':'mindatept'},axis=1)
ad_conddrug = ad_conddrug.merge(ptmindate, on = 'person_id', how = 'left').sort_values(['person_id','min_date'])

In [ ]:
# get controls

# Excluded patients: patients with dementia or dementia drug
sqlstring = """SELECT * FROM
    (SELECT DISTINCT DE.person_id FROM condition_occurrence CO
    LEFT JOIN concept cc ON CO.condition_concept_id = cc.concept_id   
    WHERE cc.concept_id in ({}))
    UNION
    (SELECT DISTINCT DE.person_id FROM drug_exposure DE
    LEFT JOIN concept cc ON DE.drug_concept_id = cc.concept_id   
    WHERE cc.concept_id in ({})) 
    """.format(','.join(neurodeg_concepts.ultconcept.astype(str).unique()),
                ','.join(demdrug_concepts.ultconcept.astype(str).unique()))
pts_with_dem_drug = pd.read_sql_query(sqlstring, con)
pts_exclude = np.unique(np.concatenate((pts_with_dem_drug.person_id, ad_conddrug.person_id)))

# then get all other patients
sqlstring = """ SELECT P.person_id, PDD.Sex, PDD.RaceEthnicity, P.birth_datetime,
                          VOE.min_date, VOE.max_date, DATEDIFF(year, P.birth_datetime, '{}') as e_age,
                          DATEADD(MONTH, -12, VOE.max_date) as mindatept
                    FROM person P 
                    LEFT JOIN {original_emr_pt_table} PDD
                        ON PDD.pt_key = P.person_source_value
                    INNER JOIN (SELECT VO.person_id, MIN((CASE WHEN VO.visit_start_date <= VO.visit_end_date THEN VO.visit_start_date
                                 ELSE VO.visit_end_date END)) as min_date, 
                                 MAX((CASE WHEN VO.visit_start_date >= VO.visit_end_date THEN VO.visit_start_date
                                 ELSE VO.visit_end_date END)) as max_date
                           FROM visit_occurrence VO
                           GROUP BY VO.person_id) VOE
                        ON VOE.person_id = P.person_id
                    WHERE (DATEDIFF(month, VOE.min_date, VOE.max_date)>=12) AND (P.person_id NOT IN ({}))
                    """.format(snapshot_date, original_emr_pt_table, ",".join(pts_exclude))
all_controls = pd.read_sql_query(sqlstring, con)

# Now extract the OMOP data for prediction models

In [ ]:
# saved index date in database for faster queries, in a temporary table called #ptdateindex

In [ ]:
# visits
sqlstring = """SELECT DISTINCT VO.visit_occurrence_id, VO.person_id, VO.visit_start_date, VO.visit_concept_id, 
                VO.care_site_id, CS.care_site_name, VO.provider_id, P.specialty_source_value
                FROM visit_occurrence VO
                LEFT JOIN care_site CS
                    on CS.care_site_id = VO.care_site_id
                LEFT JOIN provider P
                    on P.provider_id = VO.provider_id
                INNER JOIN #ptdateindex MD
                    on MD.person_id = VO.person_id"""
gvis = pd.read_sql_query(sqlstring,con)
for m in ['visit_concept_id']:
    gvis[m+'_value']=gvis[m].map(conceptToNameMap(gvis[m]))
gvis = gvis.sort_values(['person_id','visit_start_date'])
gvis.to_csv('data/cohort_visits.csv')
del gvis

In [ ]:
# conditions
sqlstring = """SELECT DISTINCT CO.person_id, CO.condition_concept_id, CO.condition_start_date, MD.mindatept
            from condition_occurrence CO
            INNER JOIN #ptdateindex MD
                on MD.person_id = CO.person_id"""
gcond = pd.read_sql_query(sqlstring,con)
for m in ['condition_concept_id']:
    gcond[m+'_value']=gcond[m].map(conceptToNameMap(gcond[m]))
gcond = gcond.sort_values(['person_id','condition_start_date'])
gcond.to_csv('data/cohort_conditions.csv')
del gcond

In [ ]:
#Measurements
sqlstring = """SELECT DISTINCT M.person_id, MD.mindatept, M.person_id, M.measurement_date, 
                M.measurement_concept_id, M.range_low, M.range_high, M.value_as_number, 
                M.value_source_value, M.unit_source_value, M.measurement_source_value
            from measurement M
            INNER JOIN #ptdateindex MD
                on MD.person_id = M.person_id"""
gmeas = pd.read_sql_query(sqlstring, con)
for m in ['measurement_concept_id']:
    gmeas[m+'_value']=gmeas[m].map(conceptToNameMap(gmeas[m]))
gmeas = gmeas.sort_values(['person_id','measurement_date'])
gmeas.to_csv('data/cohort_measures.csv')
del gmeas

In [ ]:
# Drug Exposures
sqlstring = """SELECT DISTINCT DE.person_id, DE.drug_concept_id, 
            DE.drug_type_concept_id, DE.dose_unit_concept_id, DE.drug_exposure_start_date, MD.mindatept,
            DE.drug_exposure_end_date, DE.effective_drug_dose, DE.dose_unit_source_value,
            DE.refills, DE.quantity, DE.sig, DE.route_source_value
            from drug_exposure DE
            INNER JOIN #ptdateindex MD
                on MD.person_id = DE.person_id
            --where DE.drug_exposure_start_date < MD.mindatept"""
gdrugs = pd.read_sql_query(sqlstring, con)
for m in ['drug_concept_id','drug_type_concept_id','dose_unit_concept_id']:
    gdrugs[m+'_value'] = gdrugs[m].map(conceptToNameMap(gdrugs[m]))
gdrugs = gdrugs.sort_values(['person_id','drug_exposure_start_date'])
gdrugs.to_csv('data/cohort_drugs.csv')
del gdrugs

In [ ]:
control_visits = list()
for k, g in tqdm(all_controls.groupby(np.arange(len(all_controls))//1000)):
    gvis = pd.read_sql_query("""SELECT DISTINCT VO.visit_occurrence_id, VO.person_id, VO.visit_start_date, VO.visit_concept_id
                FROM visit_occurrence VO
                WHERE VO.person_id in ({})""".format(",".join(g.person_id.astype(str).unique())), con)
    for m in ['visit_concept_id']:
        gvis[m+'_value']=gvis[m].map(conceptToNameMap(gvis[m]))
    gvis = gvis.sort_values(['person_id','visit_start_date'])
    control_visits.append(gvis)
    
control_visits = pd.concat(gvisall)
control_visits.to_csv('cohort_selection/controlvisits.csv', index = False )
del control_visits, gvis
gc.collect()

In [ ]:
# Conditions
control_conditions = list()
for k, g in tqdm(all_controls.groupby(np.arange(len(all_controls))//1000)):
    gcond = pd.read_sql_query("""SELECT DISTINCT CO.person_id, CO.condition_concept_id, CO.condition_start_date
                from condition_occurrence CO
                WHERE CO.person_id in ({})""".format(",".join(g.person_id.astype(str).unique())), con)
    for m in ['condition_concept_id']:
        gcond[m+'_value']=gcond[m].map(conceptToNameMap(gcond[m]))
    gcond = gcond.sort_values(['person_id','condition_start_date'])
    control_conditions.append(gcond)
    
control_conditions = pd.concat(control_conditions)
control_conditions.to_csv('cohort_selection/controlconditions.csv', index = False )
del control_conditions, gcond
gc.collect()

In [ ]:
# Drug Exposures
control_drugs = list()
for k, g in tqdm(all_controls.groupby(np.arange(len(all_controls))//1000)):
    sqlstring = """SELECT DISTINCT DE.person_id, DE.drug_concept_id, 
            DE.drug_type_concept_id, DE.dose_unit_concept_id, DE.drug_exposure_start_date, 
            DE.drug_exposure_end_date, DE.effective_drug_dose, DE.dose_unit_source_value,
            DE.refills, DE.quantity, DE.sig, DE.route_source_value
            from drug_exposure DE
            WHERE DE.person_id in ({})""".format(",".join(g.person_id.astype(str).unique()))
    control_drugs = pd.read_sql_query(sqlstring, con)
    for m in ['drug_concept_id','drug_type_concept_id','dose_unit_concept_id']:
        gdrugs[m+'_value'] = gdrugs[m].map(conceptToNameMap(gdrugs[m]))
    gdrugs = gdrugs.sort_values(['person_id','drug_exposure_start_date'])    
    control_drugs.append(gdrugs)
control_drugs = pd.concat(control_drugs)
control_drugs.to_csv('cohort_selection/controldrugs.csv')
del control_drugs, gdrugs
gc.collect()

In [ ]:
# MEASURES
control_measures = list()
for k, g in tqdm(all_controls.groupby(np.arange(len(all_controls))//1000)):
    print(k)
    sqlstring = """SELECT DISTINCT M.person_id, M.measurement_date, 
                M.measurement_concept_id, M.range_low, M.range_high, M.value_as_number, 
                M.value_source_value, M.unit_source_value, M.measurement_source_value
                from measurement M
                WHERE M.person_id in ({})""".format(",".join(g.person_id.astype(str).unique()))
    gmeas = pd.read_sql_query(sqlstring, con)
    for m in ['measurement_concept_id']: gmeas[m+'_value']=gmeas[m].map(conceptToNameMap(gmeas[m]))
    gmeas = gmeas.sort_values(['person_id','measurement_date'])
    control_measures.append(gmeas)
    
    if ((k%500)==0): # This was implemented to split into multiple files
        print('saving: num {}'.format(k))
        control_measures = pd.concat(control_measures)
        control_measures.to_csv('cohort_selection/controlmeasures{}.csv'.format(k//500), index = False)
        del control_measures
        gc.collect()
        control_measures = list()
control_measures = pd.concat(control_measures)
control_measures.to_csv('cohort_selection/controlmeasures{}.csv'.format(len(all_controls)//1000//500+1), index = False)

